# Formation RNAseq CEA - juin 2023

*Enseignantes : Sandrine Caburet et Claire Vandiedonck*

Session IFB : 5 CPU + 21 GB de RAM

# Part 9: Normalisation and Differential Expression analysis with DESeq2 

- 0. 1 - Setting up this R session on IFB core cluster  
- 0. 2 - Parameters to be set or modified by the user   
- 1 - Loading input data: countdata, metadata and gene info
- 2 - Modeling count distribution for each gene with `DESeq()`
- 3 - Normalisation of count distribution with `rlog` and `vst`
- 4 - Differential expression analysis with `DESeq2`   
- 5 - Saving our results for later use : RData file

---
---
## 0 - Getting started
---

### 0.1 - Setting up this R session on IFB core cluster

<em>loaded JupyterLab</em> : Version 3.2.1

In [ ]:
## Code cell 1 ##

session_parameters <- function(){
    
    jupytersession <- c(system('echo "=== Cell launched on $(date) ==="', intern = TRUE),
                        system('squeue -hu $USER', intern = TRUE))
    
    jobid <- system("squeue -hu $USER | awk '/jupyter/ {print $1}'", intern = TRUE)
    jupytersession <- c(jupytersession,
                        "=== Current IFB session size: Medium (5CPU, 21 GB) ===",
                        system(paste("sacct --format=JobID,AllocCPUS,NODELIST -j", jobid), intern = TRUE))
    print(jupytersession[1:6])
    
    return(invisible(NULL))
}

session_parameters()

__

Next we load into this R session the various tools that we will use.   
***DO NOT worry*** if you see a large red output!! It contains only a warning message, including for functions from different packages sharing the same names.

In [ ]:
## Code cell 2 ##

# list the required libraries from the CRAN repository
requiredLib <- c(
     "tidyverse",
    "magrittr",
    "matrixStats",
    "BiocManager",
    "ggplot2"
)

# list the required libraries from the Bioconductor project
requiredBiocLib <- c("DESeq2", "vsn")

# install required libraries if not yet installed
for (lib in requiredLib) {
  if (!require(lib, character.only = TRUE, quiet = TRUE)) {
    install.packages(lib, quiet = TRUE)
  }
}

for( lib in requiredBiocLib) {
  if (!require(lib, character.only = TRUE, quiet = TRUE)) {
  BiocManager::install(lib, quiet = TRUE)
  }
}

# load libraries
message("Loading required libraries")
for (lib in requiredLib) {
  library(lib, character.only = TRUE)}
for (lib in requiredBiocLib) {
  library(lib, character.only = TRUE)}

# remove variables from the R session if they are no longer necessary 
rm(lib, requiredLib, requiredBiocLib)

In [ ]:
## Code cell 3 ##   

cat("Here is my R session with the loaded packages:\n")
sessionInfo()

---
### 0.2 - Parameters to be set or modified by the user
---


- Using a full path with a `/` at the end, **define the folder** of the project as  `gohome` variable, and the folder where you work as the `myfolder` variable:

<div class="alert alert-block alert-warning"> <b> Warning on working directory: </b><br>In a Jupyter Hub and a jupyter notebook in R, by default the working directory is where the notebook is opened for the <b> fisrt time </b>. Even if you move it ton another directory, it keeps the original working diretory unless you set it again with the function <code>setwd()</code>.</div>

In [ ]:
## Code cell 4 ##

gohome <- "/shared/projects/2312_rnaseq_cea/"
gohome

myfolder <- getwd()
myfolder

- With a `/` at the end, define the path to the folder where the results of this analysis with `DESeq2` will be stored:

In [ ]:
## Code cell 5 ##

# creation of the directory, recursive = TRUE is equivalent to the mkdir -p in Unix
dir.create(paste(myfolder,"/Results/deseq2/", sep = ""), recursive = TRUE)

# storing the path to this output folder in a variable
deseq2folder <- paste(myfolder,"/Results/deseq2/", sep = "")
deseq2folder

# listing the content of the folder
print(system(paste("ls -hlt", deseq2folder), intern = TRUE) )

- Last, we specify the size of the graphical outputs that will be used for all the plots in the notebook.    
This setting could be modified at will for each plot. 

In [ ]:
## Code cell 6 ##

options(repr.plot.width = 15, repr.plot.height = 8) # for figure display in the notebook

---
---
## 1 - Loading data
---

### 1.1 - Count data and metadata
---

First we need to retrieve the counts data and the metadata about samples. As we stored those in a global `.RData` object at the end of "Pipe_08", we can simply reload all our information by openig this .Rdata object. 

In [ ]:
## Code cell 7 ##

pca1folder <- paste0(myfolder,"/Results/pca1/")
pca1folder
rdata <- paste0(pca1folder,"RawCounts_Samples.RData")
rdata
load(rdata,verbose = T)
rm(pca1folder) # not used any further

We list all the objects we have currently in our session: 

In [ ]:
## Code cell 8 ##

ls() 

We can now see it contains the two following dataframes:

- `countdata`: the dataframe containing all the read counts produced by `featureCounts` (as in the Pipe_06 notebook), ***on all 11 samples***, without the low counts genes.      
- `samples`:  the metadata dataframe containing the information about the samples, in particular the conditions of the experiment.

We will verify their content and structure.

***-> Let's start with the `samples` object containing the metadata.*** We see below it has 11 rows, one per sample.

In [ ]:
## Code cell 9 ##

samples # since we have only 11 rows, we can easily print the whole file. We could aslo have used head(samples, n=11).
str(samples)

***-> We verify now that the raw `countdata` dataframe look like what we expect*** by checking the first rows and it structure. We see below it has 12 columns, one for Geneid and the other ones for each sample, while genes are present in rows.

In [ ]:
## Code cell 10 ##

head(countdata)
str(countdata)

-> ***Most importantly**, to ensure the **metadata `samples` and `countData` dataframes have the same sample names***, we run the code below. It tests for equality in the sample names in both dataframes, in the colnames of countdata (except column 1 for gene ids) and the column "SampleName" in samples. This will return TRUE for each name, if theay are the same in both files. The `table()` function counts how many are equal. In all there are 11 samples in both files.

In [ ]:
## Code cell 11 ##

table(colnames(countdata[,-1]) == samples$SampleName, useNA = "ifany")

### 1.2 - Annotation of genes in `gene_info`
---

In the above `countdata` dataframe, we can notice the genes are only indicated with their Ensembl GeneID (http://www.ensembl.org/Mus_musculus/Info/Index). It will thus be useful to obtain further information, including gene names or coordinates.

To do so we create a `genecode`dataframe from the Gencode annotation GTF file of the mouse genome. We have already downloaded it for all users of the project in `/shared/projects/2312_rnaseq_cea/allData/Reference/` from this link: https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M32/gencode.vM32.annotation.gtf.gz. It can easily be done in bash with the `wget` command or its equivalent `wget()` fonction in R.

-> We now import it into R and look at its structure and the first rows.

In [ ]:
## Code cell 12 ##

gencode <- read_tsv("/shared/projects/2312_rnaseq_cea/allData/Reference/gencode.vM32.primary_assembly.annotation.gtf.gz",
                                     skip = 5, col_names = F, show_col_types = FALSE)


In [ ]:
## Code cell 13 ##

str(gencode)
head(gencode)

We can see above that it contains 190,516 genes in rows and 9 columns with the chromosome, the curated source (HAVANA or ENSEMBL), the type of gene, the start and end position on the chromosome and the strand.

The last column includes many information including gene_id, gene_type, gene_name, MGI_id or Havana ID. Its structure is complicated and we will parse it to sperate fields delimited by ";". In addition, we will only select annotations with the type "gene". This will be done with some R commands of the `tidyverse` package (https://www.tidyverse.org/). Do not worry with the warning message. It occured because some genes have not all fileds filled in the last column.

In [ ]:
## Code cell 14 ##

gencode %<>% filter(X3=="gene")
gencode %<>% dplyr::select(X1,X2, X4,X5,X7,X9) 
#gencode %<>% rename(chr=X1, annotation=X2, start=X4, end=X5, strand=X7)
names(gencode)[1:5] <- c("chr", "annotation", "start", "end", "strand")
gencode %<>% separate(X9, into = c("gene_id", "gene_type", "gene_name", "level", "mgi_id", "havana_gene"), sep = ";")
gencode %<>% mutate(gene_id = str_remove(gene_id, "gene_id \""),
                   gene_type = str_remove(gene_type, " gene_type \""), 
                   gene_name = str_remove(gene_name, " gene_name \""),
                   mgi_id = str_remove(mgi_id, " mgi_id \""),
                   havana_gene = str_remove(havana_gene, " havana_gene \""))
gencode %<>% mutate(gene_id = str_remove(gene_id, "\""),
                   gene_type = str_remove(gene_type, "\""), 
                   gene_name = str_remove(gene_name, "\""),
                   mgi_id = str_remove(mgi_id, "\""), 
                   havana_gene = str_remove(havana_gene, "\""))

Let's have a look at the generated file.

In [ ]:
## Code cell 15 ##

str(gencode)
head(gencode)

It is a `tibble`, a sort of dataframe object with slightly different properties. For example, it does not have row names. It is appropriate for large datasets.

The file contains now **57,010 Ensembl IDs**. 

***-> Could we only use the gene name for further analyses?***

In [ ]:
## Code cell 16 ##

table(table(gencode$gene_name))
length(unique(gencode$gene_id))
length(unique(gencode$gene_name))

As you can see, the file contains 56,860 unique gene names: 56,713 have a unique gene ID, while 3 genes have 3 different Ensembl ID and 144 genes have 2 Ensembl IDs. We will keep them all and write the parsed gene_info file on our personal directory in the Data folder.

In [ ]:
## Code cell 17 ##

write.table(gencode, file = paste0(myfolder, "/Data/gene_info.txt"), quote=F, sep="\t", row.names = F, col.names = T)

**=> Great!!** All is set now to perform normalisation and differential expression analysis with `DESeq2`.

---
---
## 2 - Modeling count distribution for each gene using library size factor and dispersion estimation with `DESeq2`
---

[**DESeq2**](https://bioconductor.org/packages/release/bioc/html/DESeq2.html) is among the best packages to perform modeling, normalisation and differential expression of quantitative omics data. It uses the **effective library size** method that assumes a majority of transcripts is not differentially expressed. DESeq2 defines a virtual reference sample by taking the median of each gene’s values across samples and then computes size factors as the median of ratios of each sample to the reference sample.

Of note, other packages using the same method that we could also use are [***EdgeR***](https://bioconductor.org/packages/release/bioc/html/edgeR.html) and [***limma***](https://bioconductor.org/packages/release/bioc/html/limma.html).

All are available on the [Bioconductor project](https://www.bioconductor.org/). 

<div class="alert alert-block alert-warning"> <b> <u>Caution on normalized and un-normalized counts: </u></b><br>
In a workflow, DESeq2 can be used twice, with two objectives:<ol>
    <li><b>generation of normalized counts, exclusively to run exploratory analyses</b>, in particular downstream visualization and analyses of results.</il>
    <li><b>differential expression analysis without using the normalized counts</b>, the model being designed to account for the library size differences internally. DE of each gene are tested using a Generalized Linear Model (GLM) of the negative bonomial family </il>
    </ol></div>

For both these objectives, there are <u>**several common steps**</u>. DESeq2 will:
- model the raw counts distribution using the ***size factors*** to account for differences in library depth
- estimate the ***gene-wise dispersion*** and shrink these estimates to generate accurate ***estimates of dispersion*** to model the counts
- fit curve to gene-wise dispersion estimates folowing a ***negative binomial model***

These common steps have been wrapped into a single function, `DESeq()`. More details on the respective steps can be found in the DESeq2 paper and in the tutorial available in the package vignette at https://bioconductor.org/packages/devel/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#the-deseqdataset. You could also type and run `?DESeq` to read more on the steps.

Below, we will see first how to generate a DESeq data object and how to model the count distributions. Then we will focus on the part generating normalized counts for downtream exploratory analyses. We will finish with the DE analysis using un-normalized counts.

### 2.1 - Create the `DESeqDataSet` object
---

DESeq2 uses an object called `DESeqDataSet` to store read counts and other intermediate calculations needed for normalisation and differential expression analysis. This is represented in the code below as ***dds*** (*d* for DESeq, *d* for Data and *s* for set). The basic idea behind this is to coerce the countdata and metadata, and any other relevant associated information, into slots of a single R object that DESeq2 can accept and work with. We do this with the `DESeqDataSetFromMatrix()` function.

#### **2.1.a - preparation of countdata matrix for DESeq2**

DESeq2 can use inputs generated from different tools. For counts data, they can be either matrices of counts, or homogeneuous dataframes with no other additional columns than the counts for each sample.

<div class="alert alert-block alert-danger"> <b> Caution on input counts: </b><br>
    <ul>
        <li> DESeq2 requires <b>raw un-normalized counts</b>, without prior normalisation of any kind, not even with log2.<il>
        <li> The counts must be <b>non-negative integers</b>. So if your matrix was generated with Salmon with pseudo counts, you have to round the values to the nearest integer.<il>
        </div>



Several steps must be carried out to model the gene count distribbutions:


- **Pre-Filter on low input genes**

For genes with low counts across samples, we will miss statistical power to detect differential expression. Keeping those genes in the dataset will also reduce the statistical power to detect other differentially expressed genes because it increases the number of multiple tests. In addition, if you work in a low-memory setting, it is preferable to reduce the size of the matrix to ease its handling,

A general rule is thus to remove genes with a total number of reads across samples below the number of samples, as we did in Pipe_08. It is recommanded to  perform this prefiltering before any normalization step. A filtering process is included in DESeq2 but we will run it before for clarity and compatibility with other normalization tools. 

Here, there are 57,010 genes analysed in our samples, if the genes with a null or a low expression were not filtered out. `DESeq2` can take care of these genes along the way, so we could have kept those. Nevertheless, here we will go on with the `countdata`dataframe, without low counts genes we got rid off in Pipe_08 by using as input the `countdata`dataframe that is already computed and contains 22,977 genes.   

In [ ]:
## Code cell 18 ##

nrow(countdata) # 22927 in counts
str(countdata)

- **Need genes as row names**

For DESeq2, counts must be provided in a matrix, or dataframe, with genes in rows put as row names, while there are only samples in columns. No additional column must be present. We reformat countdata accordingly.

In [ ]:
## Code cell 19 ##

row.names(countdata) <- countdata[,"Geneid"] # here we used the name of the column, we could also have put the index (i.e) without quotes
countdata <- countdata[,-1] # to remove the first column

We check this formating was effective.

In [ ]:
## Code cell 20 ##

head(countdata)

We run a final check on sample names that must be identical and in the same order in the coundata and samples dataframe.

In [ ]:
## Code cell 21 ##

all(names(countdata) == samples$SampleName)

#### **2.1.b - preparation of sample_info (*metadata*) matrix for DESEq2**

- **sample names**

It is recommanded for DESeq2 to put sample names as row names of the matrix, in addition to the column/variable where they already are.

In [ ]:
## Code cell 22 ##

row.names(samples) <- samples[,"SampleName"]
head(samples)

* **Factorisation**  

Categorical variables of interest to make comparisons must be formatted as factors in R for DESeq2.

We have to tell DESeq2 which information is used as a *factor*, meaning the one(s) that is going to be used as a way to separate groups for future comparisons.   
This is done by using the `factor()` function, and providing the name of the column containing this information, and the various values that this info can take.   
Feeding this factorisation into our samples dataframe changes the way this column will be read: as a factor instead of mere characters. 
   
 

In [ ]:
## Code cell 23 ##

samples$Condition <- factor(samples$Condition, levels = c("WT", "dHet", "dHetRag" ))

We can verify that this factorisation worked, by showing how many samples were placed in each group:

In [ ]:
## Code cell 24 ##

table(samples$Condition)

and by checking the levels included in that factor or checking the structure of the factor itself:

In [ ]:
## Code cell 25 ##

levels(samples$Condition)
str(samples$Condition)

  Alternatively, the `relevel()` function can also be used to clearly specify which level is used as a reference. Most of the times, it will be the WT condition.  

In [ ]:
## Code cell 26 ##

samples$Condition <- relevel(samples$Condition, ref = "WT")

In [ ]:
## Code cell 27 ##

table(samples$Condition)

If we look at our `samples`dataframe, we can now see that our `Condition`column is marked as a factor: **fct**

In [ ]:
## Code cell 28 ##

head(samples, n = 2)
str(samples)

#### **2.1.c - Create the `dds` object**

We can now create the `dds` object with 3 inputs:

- `countdata` dataframe as ar argument to the the option countData
- `samples` dataframe as an argument of the option colData
- `experimental design` to define the formula of the gene expression linear model

For a good tutorial on defining models with different type of experimental designs (one factor with 2 levels, one factor with 3 levles, two factors, two nested factors, paired samples...), see this [tutorial](https://github.com/tavareshugo/tutorial_DESeq2_contrasts/blob/main/DESeq2_contrasts.md) and companion [slides](https://docs.google.com/presentation/d/1B9zW1_F-kBqQEu4xqxIJrudYP5DecytYMRR6bY4H6aM/edit). For more complex designs see chapter 8 of limma package [userguide](https://bioconductor.org/packages/release/bioc/vignettes/limma/inst/doc/usersguide.pdf).

<div class="alert alert-block alert-danger"> <b> Caution on design: </b><br>The full design must be specified at the beginning of the analysis. It is used by all  DESeq2 functions treating samples except for the setp of the size factor estimation. The design formula should have all of the factors in your metadata that account for major sources of variation of gene expression in your experiment. This includes you factor(s) of interest. Add also any covariates (eg age or sex). You may have prior knowledge of the sources of variation (example with covariates) or you may have identified them during the QC. It is critical to include all factors in your design formula. Unfortunately, it reduces your power but without including them you may end up with inacurate results. The design formula can be changed later if you want to change the reference level or factor. </div>

Here the dataset has a nested design (condisering the double mutation or the proB state) but we are missing replicates to evaluate variation in each group.
For simplicity, we will only consider the "Condition" factor defining the "dHet", "dHetRag" and "WT" levels.

Let's create the `dds`  object with the `DESeqDataSetFromMatrix()` function dedicated to count matrices as inputs. *(Note that other function exist to import results of other R packages)*.


In [ ]:
## Code cell 29 ##

# round() is used as a security, because DESeq2 works only in integers. Read counts should be integers.
# Be careful: countData is the name of a variable for this function, and countdata is the name of our dataframe

dds <- DESeq2::DESeqDataSetFromMatrix(countData = round(countdata, digits = 0),
                              colData = samples,
                              design = ~ Condition)

We check the number of genes in the dataset

In [ ]:
## Code cell 30 ##

nrow(dds) # 22927 in counts

We can also print dds and see that the assay slot contains the raw counts that were in countdata dataframe. We can even extract it back from dds with the function `assay().`

In [ ]:
## Code cell 31 ##
dds

In [ ]:
## Code cell 32 ##
head(assay(dds)) # same as coutdata

### 2.2 - Run the DESeq analysis on the dds object
---

Next, the `DEseq` function performs the 3 following steps on the dds object we created:

    1.Estimation of size factors
    2.Estimation of dispersion
    3.Negative Binomial GLM fitting and Wald statistic 
    
For detailed explanation, please refer to the DESeq2 links above.
In this section, we will discuss the first two steps. The test part will be presented in part 4 of this notebook.

In [ ]:
## Code cell 33 ##

dds2 <- DESeq2::DESeq(dds)

Let's inspect the newly created object dds2. *(Note that in most tutorials, they often replace dds here. In fact, the dds object is completed with the results of the 3 steps. For clarity, we recalled the object dds2).*

In [ ]:
## Code cell 34 ##

dds2

The portion `assays` within `dds2` contains the expression data, still not normalized:

In [ ]:
## Code cell 35 ##

head(assay(dds2))

The `counts()` function with default parameters also returns the same raw counts stored in the assays slot.

In [ ]:
## Code cell 36 ##

head(counts(dds2))

However, if we add the argument `normalize = TRUE` it returns normalised counts obtained after dividing raw counts by the library size factor.

In [ ]:
## Code cell 37 ##

head(counts(dds2, normalize = TRUE))

Before looking at their distributions, let's comment the two main steps just performed by DESeq().

- The first setp run by DESeq() was an **estimation of size factors**. This initial step did not use the desig matrix.

We can have a look at the size factor values for each sample with the `sizeFactors()` function. Let's see how it correlates with library size given by `colSums(assay(dds2))` on raw data and with the normalized counts given by `colSums(counts(dds2, normalize = TRUE))`. We put them together in a dataframe.

In [ ]:
## Code cell 38 ##

data.frame(colData(dds2)[,c("Condition", "Tissue")],
           "sizeFactor" = sizeFactors(dds2),
           "library.size" = colSums(assay(dds2)),
           "total number after norm" = colSums(counts(dds2, normalize = TRUE)))

You can see that larger size factors correspond to samples with higher sequencing depth.

- The next step run by DESeq() was the **estimation of the dispersion.**

The dispersion models the within-group variability. It describes how much variance deviates from the mean.

We can see it with the `plotDispEsts()` function. The black dots represent the raw data of each gene. 

In [ ]:
## Code cell 39 ##

plotDispEsts(dds2)

The plot here is as expected in a typical RNASeq data set where the dispersion increases when variance incraases, but it decreases when mean increases.

Given the small sample size of most RNASeq datasets, estimation of gene variation is difficult. DESeq2 will borrow information from other genes to generate better estimates. It assumes that genes with similat expression levels have a similar variannce. The red line corresponds to the fitted curve to gene-wise dispersion estimates. This curve facilitates the identification of DE genes when sample sizes are small. In blue is the estimated dispersion for each gene toward the predicted values by the curve.

- Now consider the **distribution of raw counts and normalized counts by DESeq()**:

In [ ]:
## Code cell 40 ##

summary(counts(dds2)) # raw counts

In [ ]:
## Code cell 41 ##

summary(counts(dds2, normalize = TRUE))# normalized counts

We can plot these distribution of read counts (with a log2 transformation) using a boxplot, to verify that the read counts have indeed not yet been normalized between samples, unless we use normalize = TRUE.

In [ ]:
## Code cell 42 ##

# make a colour vector
#conditionColor <- match(samples$Condition, c("dHet", "dHetRag", "WT")) + 1
# '+1' to avoid color '1' i.e. black

# to display the two plots side by side
layout(matrix(1:2, ncol=2))

# Check distributions of samples using boxplots for the non-normalised data in log2
boxplot(log2(counts(dds2)+1), outline = FALSE,
        xlab="",
        ylab="Log2(dds2 raw counts)",
        las=2,
        col=conditionColor,
        main="Log2(dds2 raw counts)")
# Let's add a blue horizontal line that corresponds to the median
abline(h=median.default(log2(counts(dds2)+1)), col="blue")

# Check distributions of samples using boxplots
boxplot(log2(counts(dds2, normalize = TRUE)+1), outline = FALSE,
        xlab="",
        ylab="DESeq2 norm counts",
        las=2,
        col=conditionColor,
        main="DESeq2 norm counts")
# Let's add a blue horizontal line that corresponds to the median
abline(h = median(log2(counts(dds2, normalize = TRUE)+1)), col="blue")


As we can see, the log2 transformation gives a data distribution closer to the normal distribution, but does not perform well for normalizing data between samples.    

The main motive behind normalization is to change numeric values in a dataset to a common scale without distorting the differences between samples. This reduces the sparsity of the data as they are brought to a common close scale.


<div class="alert alert-block alert-danger"> <b> Caution on DESeq() norm counts: </b><br><ul> <li>For further exploratory analyses, it is not recommanded to use the normalised counts from DESeq() but one of the 2 other normalisation methods: <code>rlog</code>and <code>vst</code>. We will test the effect of both in part 3.<il>
    <li> For DE analysis, DESeq() norm counts must not be used since the linear model was already tested with the function. We will see in part 4 how to perform the DE analysis with the factor and contrats of interest.<il> </div>

---
---
## 3 - Normalization of the data with `rlog` or `vst` for vizualisation and further exploratory analyses
---

The following two methods of normalization implented in DESeq2 package are to be considered prior to exploratory analyses, like PCA, hierarchical clustering or heatmap, that will follow in the next pipeline.

### 3.1 - Data normalisation with `rlog`: Regularized-logarithm transformation
---

This function transforms the count data to the log2 scale in a way which minimizes differences between samples for rows with small counts, and which normalizes with respect to library size. For genes with high counts, rlog will not differ much from an ordinary log2 transformation.

> *In the function calls, we specified `blind = FALSE`, which means that differences between cell lines and treatment (the variables in the design) will not contribute to the expected variance-mean trend of the experiment. The experimental design is not used directly in the transformation, only in estimating the global amount of variability in the counts. For a fully **unsupervised** transformation, one can set `blind = TRUE` (which is the default).*   
[Love MI *et al.*, 2019](https://www.bioconductor.org/packages/release/workflows/vignettes/rnaseqGene/inst/doc/rnaseqGene.html#the-variance-stabilizing-transformation-and-the-rlog)

`blind=FALSE` greatly reduces the run time.

In [ ]:
## Code cell 43 ##

rlog.dds2 <- rlog(dds2, blind = FALSE)
rlog.dds2

Let's verify the effect by plotting the counts from `rlog.dds2`, side by side with the non-normalised data to see the effect of normalisation.

In [ ]:
## Code cell 44 ##

# Get log2 counts
counts.rlog.dds2 <- assay(rlog.dds2)

summary(counts.rlog.dds2) # summary for each column

In [ ]:
## Code cell 45 ##

# make a colour vector
#conditionColor <- match(samples$Condition, c("dHet", "dHetRag", "WT")) + 1
# '+1' to avoid color '1' i.e. black

# to display the two plots side by side
layout(matrix(1:2, ncol=2))

# Check distributions of samples using boxplots for the non-normalised data in log2
boxplot(log2(counts(dds2)+1), outline = FALSE,
        xlab="",
        ylab="Log2(dds2 raw counts)",
        las=2,
        col=conditionColor,
        main="Log2(dds2 raw counts)")
# Let's add a blue horizontal line that corresponds to the median
abline(h=median.default(log2(counts(dds2)+1)), col="blue")

# Check distributions of samples using boxplots for the rlog normalised data
boxplot(counts.rlog.dds2,
        xlab="",
        ylab="rlog.dds2 counts",
        las=2,
        col=conditionColor,
        main="rlog.dds2 counts")
# Let's add a blue horizontal line that corresponds to the median
abline(h=median.default(counts.rlog.dds2), col="blue")

We can also verify that the variance is not correlated any more to the average level of expression, by plotting standard deviation vs mean expression:

In [ ]:
## Code cell 46 ##

# Raw counts mean expression Vs standard Deviation (SD)
plot(rowMeans(assay(rlog.dds2)), matrixStats::rowSds(as.matrix(assay(rlog.dds2)), na.rm=TRUE), 
     main='rlog normalized read counts: sd vs mean', 
     xlim=c(-1,17),
     ylim=c(-1,5))

In [ ]:
## Code cell 47 ##

vsn::meanSdPlot(assay(rlog.dds2), ranks=FALSE)

### 3.2 - Normalization with `vst`: variance stabilizing transformation
---

Variance stabilizing transformation (VST) aims at generating a matrix of values for which variance is constant across the range of mean values, especially for low mean.

The vst function computes the fitted dispersion-mean relation, derives the transformation to apply and accounts for library size.

In [ ]:
## Code cell 48 ##

vst.dds2 <- vst(dds2, blind = FALSE)
vst.dds2

Again, we check the effect by looking at summaries and plotting the log2 counts from `vst.dds2`

In [ ]:
## Code cell 49 ##

# Get log2 counts
counts.vst.dds2 <- assay(vst.dds2)

summary(counts.vst.dds2) # summary for each column

In [ ]:
## Code cell 50 ##

# make a colour vector
#conditionColor <- match(samples$Condition, c("dHet", "dHetRag", "WT")) + 1
# '+1' to avoid color '1' i.e. black

# to display the two plots side by side
layout(matrix(1:3, ncol=3))


# Check distributions of samples using boxplots for the non-normalised data in log2
boxplot(log2(counts(dds2)+1), outline = FALSE,
        xlab="",
        ylab="Log2(dds2 raw counts)",
        las=2,
        col=conditionColor,
        main="Log2(dds2 raw counts)")
# Let's add a blue horizontal line that corresponds to the median
abline(h=median.default(log2(counts(dds2)+1)), col="blue")

# Check distributions of samples using boxplots for the rlog normalised data
boxplot(counts.rlog.dds2,
        xlab="",
        ylab="rlog.dds2 counts",
        las=2,
        col=conditionColor,
        main="rlog.dds2 counts")
# Let's add a blue horizontal line that corresponds to the median
abline(h=median.default(counts.rlog.dds2), col="blue")

# Check distributions of samples using boxplots
boxplot(counts.vst.dds2,
        xlab="",
        ylab="vst.dds2 counts",
        las=2,
        col=conditionColor,
        main="vst.dds2 counts")
# Let's add a blue horizontal line that corresponds to the median
abline(h=median.default(counts.vst.dds2), col="blue")

We also verify the decoupling of variance and mean level of expression:

In [ ]:
## Code cell 51 ##

# Raw counts mean expression Vs standard Deviation (SD)
plot(rowMeans(assay(vst.dds2)), matrixStats::rowSds(as.matrix(assay(vst.dds2)), na.rm = TRUE), 
     main = 'vst normalized read counts: sd vs mean', 
     xlim = c(5,17),
     ylim = c(-1,5))

In [ ]:
## Code cell 52 ##

vsn::meanSdPlot(assay(vst.dds2), ranks=FALSE)


We can also display the scatterplot of transformed counts from two samples:

In [ ]:
## Code cell 53 ##

df <- bind_rows(
  as_data_frame(log2(counts(dds2, normalized=TRUE)[, 1:2]+1)) %>%
         mutate(transformation = "log2(x + 1)"),
  as_data_frame(assay(rlog.dds2)[, 1:2]) %>% mutate(transformation = "rlog"),
  as_data_frame(assay(vst.dds2)[, 1:2]) %>% mutate(transformation = "vst"))
  
colnames(df)[1:2] <- c("x", "y")  

lvls <- c("log2(x + 1)", "rlog", "vst")
df$transformation <- factor(df$transformation, levels=lvls)

ggplot(df, aes(x = x, y = y)) + geom_hex(bins = 80) +
  coord_fixed() + facet_grid( . ~ transformation)  

### 3.3- Choosing the best transformation for our data for further exploratory analyses
---

You can see that both methods worked on our data, even if the output are different.   
    
The indications to guide the choice between the two methods are:

> *For genes with high counts, both the VST and the rlog will give similar result to the ordinary log2 transformation of normalized counts. For genes with lower counts, however, the values are shrunken towards a middle value. The VST or rlog-transformed data then become approximately homoskedastic (more flat trend in the meanSdPlot), and can be used directly for computing distances between samples, making PCA plots, or as input to downstream methods which perform best with homoskedastic data.  
> **Which transformation to choose?** The VST is much faster to compute and is less sensitive to high count outliers than the rlog. The rlog tends to work well on small datasets (n < 30), potentially outperforming the VST when there is a wide range of sequencing depth across samples (an order of magnitude difference). We therefore recommend the VST for medium-to-large datasets (n > 30). You can perform both transformations and compare the meanSdPlot or PCA plots generated, as described below.*  
[Love MI *et al.*, 2019](https://www.bioconductor.org/packages/release/workflows/vignettes/rnaseqGene/inst/doc/rnaseqGene.html#the-variance-stabilizing-transformation-and-the-rlog)

Both tranformation methods performed well on our dataset, but we can notice that, in the rlog normalisation, the distribution appear more centered on the median for each sample, and the variance seems less related to the mean level of expression, especially for low values. Furthermore, there are still some outliers for the vst nomalisation.  

Thus, we will keep the **rlog** transformation to go on with the the second exploratory analysis in the next notebook.

In [ ]:
## Code cell 54 ##

head(assay(rlog.dds2))

Before saving the rlog counts for further analysis, we will merge this data with the gene info stored in gencode dataframe in order to have gene names in addition to Ensembl IDs. 

In the `merge()` function, we specify both dataframes, followed by `by.x = 0` to indicate the `gene_id` key is present in the row names of the first dataframe.

In [ ]:
## Code cell 55 ##

rlog.dds2.annot <- merge(as.data.frame(assay(rlog.dds2)),
                gencode,
                by.x = 0, by.y = "gene_id",
                all.x = T, sort = F)
names(rlog.dds2.annot)[1] <- "ensemblID"

In [ ]:
## Code cell 56 ##

head(rlog.dds2.annot)

---
---
## 4 - Differential expression analysis with `DESeq2`
---

Differential expression analysis is classically the main way to analyse expression data from a quantitative point of view.   
This approach relies on statistical tests to identify the genes that present a level of expression that is significantly different  from one condition to another, or between several conditions.  
The genes are therefore considered as Differentially Expressed, and you'll often see the abreviation *DE genes*.   

To identify those DE genes, we need to:
- define the comparison(s) that we want to perform, by building a *contrast*
- run the analysis of expression data using this constrast. 

Tools such as `DESeq2` will :
- take into account the contrast to define the groups to compare, 
- compute the within-group and the between-group variability, 
- run a statistical test on each gene to look for statistically-significant differences of expression, 
- correct the resulting p-value to account for multiple testing
- return the list of genes with an adjusted p-value below a given threshold

### 4.1 - Building the results table
---

The `results` table is a dataframe that contains our differentially expressed genes, their p-values, whether the genes are upregulated or downregulated, the number of outliers, etc.  

`summary()` gives us the genes that are up and down regulated in our condition under study, as well as low count genes and outliers. This generates summary tarries for the data. You can read about more about the results function by looking up `?results`

* **Choose the comparison to perform**

We begin by defining the contrast, that is the comparison that we want to perform.  
When geneating dds2, we specified the WT group as a reference. We can see that the current model design will test the contrats between either dHet or dHetRag versus the WT group.

In [ ]:
## Code cell 57 ##

resultsNames(dds2)

We always can define new contrats.

  
Here we choose to compare **the ALL leukemic cells to the dHet-Rag non leukemic ones**: 

In [ ]:
## Code cell 58 ##

dHet_dHetRag <- c("Condition", "dHet", "dHetRag")

* **Perform the comparison between groups**

Now we can run `DESeq2` for this comparison. It will not work on the normalised rlog.dds2, but on the not-yet normalised dds2 : indeed the normalisation step has to take into account the inter-group variability, and the intra-group variability to be correct, meaning that... the groups have to be defined first!   
Now that `DESeq2` knows which groups we want to compare, thanks to the `contrast` defined above, it will run the normalisation on the fly, and identify the genes that are significantly differentially expressed (DE genes) according to that specific normalisation.  

In [ ]:
## Code cell 59 ##

res_dHet_dHetRag <- results(dds2, contrast = dHet_dHetRag)
summary(res_dHet_dHetRag)

Here, we found very few outliers, and we have 7.8% of low counts genes.   
The threshold used to consider a gene as significantly differentially expressed is a adjusted p-value of 0.1. Adjusted p-value means that a correction has been performed on the raw p-value, to take into consideration that the statistical test have been reiterated many times, as many as the number of genes. On such a high number of tests, numerous will give a "significant result" only by chance.   
The most-commonly used p-value correction is the **Benjamini-Hochberg method**, it is the one used by default by `DESeq2`. You could specify another one in the argument of the function.  

We obtain 4714 significantly up-regulated genes + 3710 significantly down-regulated genes. LFC stands for Log2 Fold Change, that is the ratio of the expression of a given gene in the test conditon on its expression in the Reference condition, in log2 scale. Therefore, a log2 FC of 1 means that the gene is 2-times more expressed in the test samples versus the reference samples.

*For information, if we run the comparison between `dHet`and `WT` cells, we obtain:*    
    out of 22927 with nonzero total read count   
    adjusted p-value < 0.1   
    LFC > 0 (up)       : 3309, 14%   
    LFC < 0 (down)     : 3542, 15%   
    outliers [1]       : 29, 0.13%   
    low counts [2]     : 2223, 9.7%   
    (mean count < 2)   
    [1] see 'cooksCutoff' argument of ?results   
    [2] see 'independentFiltering' argument of ?results

### 4.2 - Working with a different threshold: alpha 0.05

By default, the result function uses an adjusted p-value (FDR) cutoff of 0.1. If any other p value is prefered, `alpha` should be set to that value, as shown below where we set `alpha=0.05`.

In [ ]:
## Code cell 60 ##

res2_dHet_dHetRag <- results(dds2, contrast = dHet_dHetRag,  alpha = 0.05)
summary(res2_dHet_dHetRag)

We can now count the number of differentially expressed genes with adjusted p-values were less than 0.05. na.rm=TRUE removes any row with missing information.

In [ ]:
## Code cell 61 ##

sum(res2_dHet_dHetRag$padj < 0.05, na.rm = TRUE)
summary(res2_dHet_dHetRag)
# 7140

So out of the 22927 filtered genes, 7140 have adjusted p-values below 0.05, with 4044 significantly up-regulated genes + 3096 significantly down-regulated genes. Let’s go ahead and select them.

In [ ]:
## Code cell 62 ##

res2_dHet_dHetRag_sig_genes <- subset(res2_dHet_dHetRag, padj < 0.05)
dim(res2_dHet_dHetRag_sig_genes)

## 7140     6

We then order the selected genes by the smallest p value:

In [ ]:
## Code cell 63 ##

res2_dHet_dHetRag_sig_ranked <- res2_dHet_dHetRag_sig_genes[order(res2_dHet_dHetRag_sig_genes$pvalue),]
dim(res2_dHet_dHetRag_sig_ranked)
head(res2_dHet_dHetRag_sig_ranked)
## [1] 7140    6



As you can see above, our current list only contains Ensembl GeneID and no genes names. 

We will thus merge DE genes with the gene info stored in gencode dataframe.  We keep only the 7140 genes that were present in the DE results by using all.x = T.

In [ ]:
## Code cell 64 ##

res2_dHet_dHetRag_sig_ranked_annot <- merge(as.data.frame(res2_dHet_dHetRag_sig_ranked), gencode, 
                                            by.x = 0, by.y = "gene_id",
                                            all.x = T, sort = F)

In [ ]:
## Code cell 65 ##

str(res2_dHet_dHetRag_sig_ranked_annot)

For further vizualization and analyses of the data, we may need to have the Ensembl gene ID as row names.

In [ ]:
## Code cell 66 ##

names(res2_dHet_dHetRag_sig_ranked_annot)[1] <- "ensemblID"
head(res2_dHet_dHetRag_sig_ranked_annot, n = 20)

You can see that the top DE gene is Ighj1 with an adjusted pvalue of 1.7e-243 and we can extract it from the results table.

In [ ]:
## Code cell 67 ##

subset(res2_dHet_dHetRag_sig_ranked_annot, gene_name == "Ighj1")

Finally, we write the differentially expressed gene list to a tabulated txt file, in the deseq2folder that we decided at first:

In [ ]:
## Code cell 68 ##

write.table(as.data.frame(res2_dHet_dHetRag_sig_ranked_annot),
            file = paste0(deseq2folder,"DESeq2_significant_genes-0_05.tsv"),
            sep = "\t", quote = FALSE, col.names = TRUE)




---
---
## 5 - Saving our results for later use: RData file
---

We can save all relevant R objects created in this session in a single R object.   
This will help us to reload our results without having to run the same commands.   

In [ ]:
## Code cell 69 ##

print(ls())

and we save our info in a single RData object in our output folder:

In [ ]:
## Code cell 70 ##

save(rlog.dds2.annot, res2_dHet_dHetRag_sig_ranked_annot, gencode, samples, file = paste0(deseq2folder, "deseq2.RData"))

---
___


Now we go on with the visualization of normalized read counts in a full exploratory analysis.  
  
**=> Step 10: Exploratory analysis of normalized data** 

The jupyter notebook used for the next session will be *Pipe_10-R403-Normcounts-exploratory-analysis-II.ipynb*    
Let's retrieve it in our directory, in order to have a private copy to work on:   

In [ ]:
## Code cell 71 ##   

myfolder
file.copy("/shared/projects/2312_rnaseq_cea/pipeline/Pipe_10-R403-Normcounts-exploratory-analysis-II.ipynb", myfolder)




**Save executed notebook**

To end the session, save your executed notebook in your `run_notebooks' folder. **Adjust the name with yours** and reformat as code cell to run it.

In [ ]:
## Code cell 72 ##   

# creation of the directory, recursive = TRUE is equivalent to the mkdir -p in Unix
dir.create(paste0(myfolder,"/run_notebooks"), recursive = TRUE)

runfolder <- paste0(myfolder,"/run_notebooks")
       
file.copy(paste0(myfolder, "/Pipe_09-R403-DESeq2-normalisation-DE.ipynb"), runfolder)


<div class="alert alert-block alert-success"><b>Success:</b> Well done! You now know how to perform a normalisation and a differential expression analysis using DESeq2.<br>
Don't forget to save you notebook and export a copy as an <b>html</b> file as well <br>
- Open "File" in the Menu<br>
- Select "Export Notebook As"<br>
- Export notebook as HTML<br>
- You can then open it in your browser even without being connected to the server! 
</div>

---
---

## Useful commands
<div class="alert alert-block alert-info"> 
    
- <kbd>CTRL</kbd>+<kbd>S</kbd> : save notebook<br>    
- <kbd>CTRL</kbd>+<kbd>ENTER</kbd> : Run Cell<br>  
- <kbd>SHIFT</kbd>+<kbd>ENTER</kbd> : Run Cell and Select Next<br>   
- <kbd>ALT</kbd>+<kbd>ENTER</kbd> : Run Cell and Insert Below<br>   
- <kbd>ESC</kbd>+<kbd>y</kbd> : Change to *Code* Cell Type<br>  
- <kbd>ESC</kbd>+<kbd>m</kbd> : Change to *Markdown* Cell Type<br> 
- <kbd>ESC</kbd>+<kbd>r</kbd> : Change to *Raw* Cell Type<br>    
- <kbd>ESC</kbd>+<kbd>a</kbd> : Create Cell Above<br> 
- <kbd>ESC</kbd>+<kbd>b</kbd> : Create Cell Below<br> 

<em>  
To make nice html reports with markdown: <a href="https://dillinger.io/" title="dillinger.io">html visualization tool 1</a> or <a href="https://stackedit.io/app#" title="stackedit.io">html visualization tool 2</a>, <a href="https://www.tablesgenerator.com/markdown_tables" title="tablesgenerator.com">to draw nice tables</a>, and the <a href="https://medium.com/analytics-vidhya/the-ultimate-markdown-guide-for-jupyter-notebook-d5e5abf728fd" title="Ultimate guide">Ultimate guide</a>. <br>
Further reading on JupyterLab notebooks: <a href="https://jupyterlab.readthedocs.io/en/latest/user/notebook.html" title="Jupyter Lab">Jupyter Lab documentation</a>.<br>   
</em>    
 
</div>

Bénédicte Noblet - 05-07 2021   
Sandrine Caburet et Claire Vandiedonck - 02-06 2023   
with adaptations from https://bioinformatics-core-shared-training.github.io/RNAseq_November_2020_remote/html/02_Preprocessing_Data.html, https://rpubs.com/adoughan/778146 , https://github.com/hbctraining/DGE_workshop_salmon_online/tree/master/lessons, https://www.bioconductor.org/packages/release/workflows/vignettes/rnaseqGene/inst/doc/rnaseqGene.html#the-variance-stabilizing-transformation-and-the-rlog and  https://bioconductor.org/packages/devel/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#the-deseqdataset

Maj 09/06/2023 par @CVandiedonck   